# <center>  **Data Wrangling** 

### <center> **Data Collection**

Goal: Organize your data to streamline the next steps of your capstone

Time estimate: 1-2 hours

<center> Data Loading

Importing Packages

In [87]:
# Import all the Necessary Packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

from statsmodels.stats.outliers_influence import variance_inflation_factor

Importing the data

In [45]:
# Import the data and see what it looks like
path = r'C:\Users\jdrel\OneDrive\Documents\Data_Science\Springboard\Capstone-2\data\raw\kddcup.data_10_percent'
data = pd.read_csv(path)

### <center> **Data Definition**

<center> Column Names

The column names can be found in the about section on the data's webpage: [KDD Dataset](https://www.kaggle.com/datasets/slashtea/kdd-cyberattack?resource=download)

In [46]:
# I copy and pasted the column names into this list
data.columns = ["duration", "protocol_type", "service", "flag", "src_bytes",
                "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
                "num_failed_logins", "logged_in", "num_compromised",
                "root_shell", "su_attempted", "num_root", "num_file_creations",
                "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login",
                "is_guest_login", "count", "srv_count", "serror_rate","srv_serror_rate",
                "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
                "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
                "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", 
                "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "labels"]
data.head(3)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.


Save Data with the column names

In [17]:
# Store the path in a variable so that it is easy to use
path =  r'C:\Users\jdrel\OneDrive\Documents\Data_Science\Springboard\Capstone-2\data\interim\KDD Data.csv'
# Save the data in a csv file
data.to_csv(path)

## <center> **Data Wrangling**

Gain an overview of the dataset.

In [ ]:
# Report exploring all of the variables in the data
# Dark Mode is enabled for my eyes
profile = ProfileReport(data, 
                        title = 'Network Intrusion Exploration',
                        dark_mode = True)
profile

## <center>**Multi-Co-linearity**

The most glaring problem from the profile report is the amount of variables that have high correlations with each other. This is because a sizable portion of the columns measure either similar things or the flip version of them. For instance "srv_rerror_rate" and "dst_host_srv_rerror_rate" are measuring the rate at which a packet is rejected but each columns measures different points on the route. There are many different examples of highly correlated features that can not all coexist in the dataset without making it impossible to not overfit a training model or determine which variables are mechanisms of action.

One of the types of analyses that this data lent itself to was clustering. With Multi-Colinearity there may be some artificial clusters since some of the data would be overrepresented by variables that repeat the same information making it appear as though the two observations are close together. As such we need to perform some feature selection so that we can be sure that the clustering algorithms is not capturing any duplicates.


In [69]:
# Only use the X data so that it is easy to test for multicolinearity
X = data.drop('labels', axis = 1)

# Find the categorical columns that need to be made numerical for analysis
cat_cols = list(X.select_dtypes(include = 'object').columns)

x_num_data = pd.get_dummies(X, columns = cat_cols, 
                        # When testing for multi-co-linearity it is important to drop one of the dummies
                        # so that that column doesn't get flagged
                            drop_first = True)

# Create the scaler object
scaler = StandardScaler()
# fit the scaler to the dataset
scaler.fit(x_num_data)
# Scale the dataset
x_num_data_scaled = pd.DataFrame(scaler.transform(x_num_data))

In [148]:
# Create a list vif that stores all of the vif values for each of the features
#vif = [variance_inflation_factor(x_num_data_scaled, i) for i in range(len(x_num_data_scaled.columns))]
# Bind the features to their vif value for analysis, only keep the features with vif>2
vif_dict = {x_num_data.columns[i]:vif[i] for i in range(len(vif)) if vif[i] < 3}

In [149]:
print(f"{len(vif_dict)} out of {len(x_num_data.columns)} features have a vif < 5")
sorted_vif_dict_keys = [k for k, v in sorted(vif_dict.items(), key=lambda item: item[1])]
print(sorted_vif_dict)
print(sorted(list(vif_dict.values()))) 

21 out of 115 features have a vif < 5
['su_attempted', 'flag_S3', 'diff_srv_rate', 'dst_host_srv_diff_host_rate', 'flag_RSTOS0', 'service_Z39_50', 'service_netstat', 'service_ctf', 'service_kshell', 'service_name', 'service_http_443', 'service_exec', 'service_netbios_dgm', 'service_ldap', 'service_pop_2', 'service_link', 'service_netbios_ns', 'service_efs', 'service_daytime', 'hot', 'service_hostnames', 'service_login', 'service_supdup', 'service_ssh', 'service_nnsp', 'service_uucp', 'service_vmnet', 'service_klogin', 'service_uucp_path', 'service_bgp', 'service_sunrpc', 'service_netbios_ssn', 'service_mtp', 'service_courier', 'service_nntp', 'service_printer', 'service_whois', 'service_sql_net', 'service_rje', 'service_shell', 'service_echo', 'service_systat', 'service_iso_tsap', 'service_discard', 'service_domain', 'logged_in', 'service_imap4', 'service_gopher', 'service_remote_job', 'service_csnet_ns', 'flag_S2', 'service_time', 'service_pop_3', 'dst_host_rerror_rate', 'dst_host_dif

This result is a major problem. While it may have seemed as though there were a lot of features to work with, instead most of the features are multicolinear and will need to be dropped. The first step is to see how many of the multicolinear variables are from dummy columns vs. the initial features.

In [140]:
# Define X and y
X = x_num_data_scaled
y = np.where(data['labels'] == 'normal.', 1, 0)

# Create Lasso model
lasso = Lasso()

# Define hyperparameter grid
params = {'alpha': np.linspace(.005, 10, 20)}

# Perform grid search
grid_search = GridSearchCV(estimator=lasso, param_grid=params, cv=8)
grid_search.fit(X, y)

# Print best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)


Best hyperparameters:  {'alpha': 0.005}


In [163]:
# Find the coefficients of lasso regularization
lasso = Lasso(alpha = .005)
lasso.fit(X,y)
lasso_dict = {x_num_data.columns[i]:lasso.coef_[i] for i in range(len(x_num_data.columns)) if list(lasso.coef_)[i] != 0}
lasso_features = [x_num_data.columns[i] for i in range(len(x_num_data.columns)) if list(lasso.coef_)[i] != 0]
print(lasso_dict)

{'wrong_fragment': -0.035665562338885966, 'hot': -0.02789990888982136, 'logged_in': 0.05039148892050847, 'is_guest_login': 0.02105738110702302, 'count': -0.1039681421071592, 'serror_rate': -0.002336134541103318, 'srv_serror_rate': -0.0007324790813765419, 'same_srv_rate': 0.067653890310172, 'dst_host_count': -0.022887906185011155, 'dst_host_same_src_port_rate': -0.02705945930892925, 'dst_host_srv_diff_host_rate': -0.0027980354781513934, 'dst_host_serror_rate': -0.004419873170670419, 'protocol_type_udp': 0.13353651456915858, 'service_auth': 0.004893524955779914, 'service_finger': 0.010930662706795692, 'service_ftp_data': 0.03178881707503904, 'service_http': 0.1621148556277428, 'service_other': 0.006298177249483122, 'service_private': -0.011537354084874236, 'service_smtp': 0.064833067950171, 'service_urp_i': 0.01932678856398204, 'flag_RSTR': -0.0072009971825016785}


In [161]:
lasso_df_features = x_num_data[lasso_features]

Now that we have a dataframe that is not supposed to have a colinearity problem we can test this one more time with the variance inflation score.

In [165]:
# Create a list vif that stores all of the vif values for each of the features
#vif2 = [variance_inflation_factor(lasso_df_features, i) for i in range(len(lasso_df_features.columns))]
# Bind the features to their vif value for analysis, only keep the features with vif>2
vif2_dict = {lasso_df_features.columns[i]:[vif2[i], v]  for i,v in enumerate(lasso_dict.values())}
print(vif2_dict)
len(vif2_dict)

{'wrong_fragment': [1.100787267567862, -0.035665562338885966], 'hot': [3.4981230144564823, -0.02789990888982136], 'logged_in': [12.931801056683987, 0.05039148892050847], 'is_guest_login': [3.6502578879738237, 0.02105738110702302], 'count': [48.235832137684106, -0.1039681421071592], 'serror_rate': [674.0203383887539, -0.002336134541103318], 'srv_serror_rate': [407.27350534384357, -0.0007324790813765419], 'same_srv_rate': [93.32800052917462, 0.067653890310172], 'dst_host_count': [29.37531689765294, -0.022887906185011155], 'dst_host_same_src_port_rate': [52.87342550920837, -0.02705945930892925], 'dst_host_srv_diff_host_rate': [1.4577261915414559, -0.0027980354781513934], 'dst_host_serror_rate': [500.7841010322061, -0.004419873170670419], 'protocol_type_udp': [5.4275249369190695, 0.13353651456915858], 'service_auth': [1.0348054931334085, 0.004893524955779914], 'service_finger': [1.0815952196882133, 0.010930662706795692], 'service_ftp_data': [1.809215631134741, 0.03178881707503904], 'servic

22

In [ ]:
sns.heatmap()

In [154]:
usable1 = [key for key, value in vif_dict.items() if value < 4]
usable2 = [key for key, value in vif2_dict.items() if value < 4]
print(len(usable1), sorted(usable1))
print(len(usable2), sorted(usable2))

21 ['diff_srv_rate', 'dst_bytes', 'dst_host_count', 'dst_host_srv_diff_host_rate', 'duration', 'flag_RSTOS0', 'flag_S3', 'land', 'num_access_files', 'num_failed_logins', 'num_file_creations', 'num_shells', 'root_shell', 'service_X11', 'service_pm_dump', 'service_tftp_u', 'src_bytes', 'srv_diff_host_rate', 'su_attempted', 'urgent', 'wrong_fragment']
11 ['dst_host_srv_diff_host_rate', 'flag_RSTR', 'hot', 'is_guest_login', 'service_auth', 'service_finger', 'service_ftp_data', 'service_other', 'service_smtp', 'service_urp_i', 'wrong_fragment']
